In [ ]:
!pip install torch torchvision transformers timm matplotlib seaborn sklearn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from typing import Tuple
import random
from PIL import Image
import timm
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, f1_score, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np



# Data loader class that applies data augmentation, change to grayscale and normalize all images in the dataset 1
class CovidDataloader:
    def __init__(self):
        dataset = "Pre_Train"
        self.train_dir = f"/content/drive/MyDrive/new_dataset/{dataset}/Train"
        self.val_dir = f"/content/drive/MyDrive/new_dataset/{dataset}/Val"
        self.test_dir = f"/content/drive/MyDrive/new_dataset/{dataset}/Test"

        def grayscale_to_rgb(image):
            return Image.merge('RGB', (image, image, image))

        self.train_transform = transforms.Compose([
            transforms.Grayscale(),
            transforms.Resize((224, 224)),
            transforms.Lambda(grayscale_to_rgb),  # Use grayscale values for all 3 channels for train set images

            # Apply data augmentation Random Horizontal and Vetrical Filp, Rotation by 15 degress, translation, scale, shear and normalize the images in train set
            transforms.RandomApply([
                transforms.RandomRotation(15),
                transforms.RandomHorizontalFlip(),
                transforms.RandomVerticalFlip(),
                transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),
                transforms.RandomAffine(degrees=0, scale=(0.9, 1.1)),
                transforms.RandomAffine(degrees=0, shear=10)
            ], p=0.5),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])

        self.val_test_transform = transforms.Compose([
            transforms.Grayscale(),
            transforms.Resize((224, 224)),
            transforms.Lambda(grayscale_to_rgb), # Use grayscale values for all 3 channels for test and validation sets
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])

        self.train_dataset = datasets.ImageFolder(root=self.train_dir, transform=self.train_transform)
        self.val_dataset = datasets.ImageFolder(root=self.val_dir, transform=self.val_test_transform)
        self.test_dataset = datasets.ImageFolder(root=self.test_dir, transform=self.val_test_transform)

    def get_loaders(self, batch_size: int = 32, num_workers: int = 2):
        train_loader = DataLoader(self.train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, drop_last=True)
        val_loader = DataLoader(self.val_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, drop_last=True)
        test_loader = DataLoader(self.test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, drop_last=True)

        return train_loader, val_loader, test_loader



dataloader = CovidDataloader()
train_loader, val_loader, test_loader = dataloader.get_loaders(batch_size=64)

print(f"Length of training dataset: {len(dataloader.train_dataset)}")
print(f"Length of validation dataset: {len(dataloader.val_dataset)}")
print(f"Length of test dataset: {len(dataloader.test_dataset)}")


Length of training dataset: 3840
Length of validation dataset: 956
Length of test dataset: 1199


In [ ]:
# Intialize the pre-trained (imagenet checkpoint) EfficientNetV2 model from timm library

model = timm.create_model('tf_efficientnetv2_b0', pretrained=True, num_classes=3)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/28.8M [00:00<?, ?B/s]

EfficientNet(
  (conv_stem): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNormAct2d(
    32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): ConvBnAct(
        (conv): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNormAct2d(
          16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (aa): Identity()
        (drop_path): Identity()
      )
    )
    (1): Sequential(
      (0): EdgeResidual(
        (conv_exp): Conv2dSame(16, 64, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (bn1): BatchNormAct2d(
          64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (aa): Identity(

In [ ]:

# Use the learning rate and weight decay used by the model for its imagenet fine-tuning
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=4e-3, weight_decay=1e-5)


writer = SummaryWriter()


num_epochs = 10

#  Train Loop for 10 epochs, train model on train set and validation set
for epoch in range(num_epochs):
    print(f"Starting epoch {epoch}/{num_epochs - 1}")
    model.train()
    running_loss = 0.0
    running_corrects = 0

    for batch_idx, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        preds = torch.max(outputs, 1)[1]
        running_corrects += torch.sum(preds == labels.data)


    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = running_corrects.double() / len(train_loader.dataset)

    model.eval()
    val_loss = 0.0
    val_corrects = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            preds = torch.max(outputs, 1)[1]
            val_corrects += torch.sum(preds == labels.data)

    val_loss /= len(val_loader.dataset)
    val_acc = val_corrects.double() / len(val_loader.dataset)

    writer.add_scalars('Loss', {'train': epoch_loss, 'val': val_loss}, epoch)
    writer.add_scalars('Accuracy', {'train': epoch_acc, 'val': val_acc}, epoch)

    print(f'Epoch {epoch}/{num_epochs - 1}, Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

writer.close()




Starting epoch 0/9


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch 0/9, Train Loss: 1.1924, Train Acc: 0.6615, Val Loss: 0.4698, Val Acc: 0.7542
Starting epoch 1/9
Epoch 1/9, Train Loss: 0.5108, Train Acc: 0.7964, Val Loss: 0.3414, Val Acc: 0.8002
Starting epoch 2/9
Epoch 2/9, Train Loss: 0.4645, Train Acc: 0.8242, Val Loss: 0.2960, Val Acc: 0.8107
Starting epoch 3/9
Epoch 3/9, Train Loss: 0.3969, Train Acc: 0.8427, Val Loss: 0.4723, Val Acc: 0.7605
Starting epoch 4/9
Epoch 4/9, Train Loss: 0.3719, Train Acc: 0.8513, Val Loss: 0.2735, Val Acc: 0.8452
Starting epoch 5/9
Epoch 5/9, Train Loss: 0.3258, Train Acc: 0.8758, Val Loss: 0.4701, Val Acc: 0.7678
Starting epoch 6/9
Epoch 6/9, Train Loss: 0.3500, Train Acc: 0.8589, Val Loss: 0.2505, Val Acc: 0.8410
Starting epoch 7/9
Epoch 7/9, Train Loss: 0.3275, Train Acc: 0.8747, Val Loss: 0.4133, Val Acc: 0.7793
Starting epoch 8/9
Epoch 8/9, Train Loss: 0.2877, Train Acc: 0.8878, Val Loss: 0.2347, Val Acc: 0.8410
Starting epoch 9/9
Epoch 9/9, Train Loss: 0.2691, Train Acc: 0.8958, Val Loss: 0.3128, Val A

In [ ]:

model.eval()
all_preds = []
all_labels = []

# Evaluate trained model on test set
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        preds = torch.max(outputs, 1)[1]
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())


# Compute Evaluation Metrics and confusion matrix

test_acc = accuracy_score(all_labels, all_preds)
print(f'Test Accuracy: {test_acc:.4f}')


f1 = f1_score(all_labels, all_preds, average='weighted')
print(f'F1 Score: {f1:.4f}')


class_report = classification_report(all_labels, all_preds, target_names=['COVID', 'Non-COVID', 'Normal'])
print('Classification Report:')
print(class_report)


conf_matrix = confusion_matrix(all_labels, all_preds)


plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['COVID', 'Non-COVID', 'Normal'], yticklabels=['COVID', 'Non-COVID', 'Normal'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


<pre>

Test Accuracy: 0.8863

F1 Score: 0.8843

Classification Report:

              precision    recall  f1-score   support

      Normal       0.87      0.99      0.92       423
   Non-COVID       0.89      0.86      0.88       398
       COVID       0.92      0.78      0.84       331

    accuracy                           0.89      1152
   macro avg       0.89      0.88      0.88      1152
weighted avg       0.89      0.89      0.88      1152

</pre>



In [ ]:

torch.save(model.state_dict(), '/content/drive/MyDrive/EfficientNetV2/new_model.pth')